# Exercise2

Gale-Shapley Algorithmを実装する. 詳細: https://github.com/OyamaZemi/exercises2016/tree/master/ex02

## imports

In [34]:
using DataStructures
include("matching_tools.jl")

_randperm2d! (generic function with 1 method)

## Argsort

まず, 1次元のargsort関数を実装する

In [2]:
function argsort{T<:Int}(array::AbstractVector{T})
    #=1次元配列をArgument Sortする
    
    配列をSortした後, その要素が元あった場所のIndexを返す
    例: [1, 3, 4, 2, 0] -> [5, 1, 4, 2, 3]
    タイの順位には非対応
    =#
    
    array_with_index = Array{T}(length(array), 2)
    for i in 1:length(array)
        array_with_index[i, 1] = array[i]
        array_with_index[i, 2] = i
    end
    
    sorted_array_with_index = sortrows(array_with_index, by=x->x[1])
    return sorted_array_with_index[:, 2]
end

array = [1, 3, 4, 2, 0]
println(argsort(array))

[5,1,4,2,3]


次に, 2次元に対応させる

In [3]:
function argsort{T<:Int}(array::AbstractArray{T, 2})
    #=2次元配列をArgument Sortする
    
    それぞれの行毎にargsortする.
    例: [1 3 4 2 0;
        　　　1 2 0 4 3]
    
    　->　[5 1 4 2 3;
        　　　3 1 2 5 4]
    =#
    
    row = size(array)[1]
    col = size(array)[2]
    argsorted = Array{T}(size(array))
    for i in 1:row
        out = argsort(squeeze(array[i, :], 1))
        for j in 1:col
            argsorted[i, j] = out[j]
        end
    end
    
    return argsorted
end

array = [1 3 4 2 0; 1 2 0 4 3]
println(argsort(array))

[5 1 4 2 3
 3 1 2 5 4]


## 1 to 1 Gale Shapley

を書く. ここでは man-optimal matchingを出力する.

In [30]:
function gale_shapley{T<:Int64}(m_prefs::AbstractArray{T, 2}, f_prefs::AbstractArray{T, 2})
    # 第1次元（行）のサイズ = 男, 女の人数 を取得
    m_size = size(m_prefs, 1)
    f_size = size(f_prefs, 1)
    
    # 仮マッチング済相手を入れる（0をunmatch）
    m_matched = zeros(Int64, m_size)
    f_matched = zeros(Int64, f_size)
    
    # 未処理の男を入れておくスタック
    stack = Stack(Int)
    for i in 0:m_size-1
        push!(stack, m_size-i)
    end
    
    # 女の選好を[男1の順位, 男2の順位,...] -> [1位の男の番号, 2位の男の番号,...] に変える
    male_rankings = argsort(f_prefs)
    
    while length(stack) != 0
        male = pop!(stack)
        
        for i in 1:f_size+1
            # 順位表が終わりまで来たら探索終了
            if m_prefs[male, i] == 0
                break
            end
            
            female = m_prefs[male, i]
            
            # 女性が誰ともマッチしていない場合, 女性にとって男性がacceptableならマッチ
            if f_matched[female] == 0 && male_rankings[female, male+1] < male_rankings[female, 1]
                m_matched[male] = female
                f_matched[female] = male
                break
            # 誰かとマッチしている場合, 男性が現在のパートナーよりもランクが高ければマッチ
            else
                current_partner = f_matched[female]
                if male_rankings[female, male+1] < male_rankings[female, current_partner+1]
                    m_matched[male] = female
                    f_matched[female] = male
                    m_matched[current_partner] = 0
                    push!(stack, current_partner)
                end
            end
        end
    end
    
    return m_matched, f_matched
end

gale_shapley (generic function with 1 method)

In [36]:
m_prefs = [2 1 3 4 0;　1 3 2 4 0; 1 3 4 2 0]
f_prefs = [2 1 3 0; 1 3 2 0; 1 3 2 0; 2 3 1 0]
println(m_prefs)
println(f_prefs)
println(gale_shapley(m_prefs, f_prefs))

[2 1 3 4 0
 1 3 2 4 0
 1 3 4 2 0]
[2 1 3 0
 1 3 2 0
 1 3 2 0
 2 3 1 0]
([2,1,3],[2,1,3,0])


In [39]:
srand(613)
m_prefs2, f_prefs2 = random_prefs(4, 3, allow_unmatched=true)
m_prefs2 = m_prefs2'
f_prefs2 = f_prefs2'
println(m_prefs2)
println(f_prefs2)
println(gale_shapley(m_prefs2, f_prefs2))

[2 3 0 1
 2 0 3 1
 1 2 3 0
 1 0 2 3]
[3 2 0 1 4
 4 1 2 0 3
 4 3 2 1 0]
([2,0,1,0],[3,1,0])
